# The following code was trained on Pytorch CUDA noteboook device with 2 NVIDIA T4 GPUS and 2vCPUs and 7.5GB RAM

## Sparsification using custom training pipeline

In [2]:
# !pip install sparseml[torch,torchvision]

In [1]:
import torch
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from sparseml.pytorch.models import resnet50
from sparseml.pytorch.datasets import ImagenetteDataset, ImagenetteSize
from sparseml.pytorch.optim import ScheduledModifierManager

# Model creation
NUM_CLASSES = 10  # number of Imagenette classes
model = resnet50(pretrained=True, num_classes=NUM_CLASSES)

# Dataset creation
batch_size = 64
train_dataset = ImagenetteDataset(train=True, dataset_size=ImagenetteSize.s320, image_size=224)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, num_workers=8)

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)

# Loss setup
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=10e-6, momentum=0.9)

# Recipe - in this case, we pull down a recipe from the SparseZoo for ResNet-50
# This can be a be a path to a local file
recipe_path = "zoo:cv/classification/resnet_v1-50/pytorch/sparseml/imagenet/pruned95_quant-none?recipe_type=original"

# SparseML Integration
manager = ScheduledModifierManager.from_yaml(recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

# Training Loop
for epoch in range(manager.max_epochs):
    running_loss = 0.0
    running_corrects = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs, _ = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    print("Training Loss: {:.4f} Acc: {:.4f}".format(epoch_loss, epoch_acc))

manager.finalize(model)

downloading imagenette ImagenetteSize.s320:   0%|          | 0.00/325M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


cuda


No default recipe recipe_original found, falling back to first listed recipe recipe_transfer_classification.md


2024-01-22 22:33:48 sparseml.core.logger INFO     Logging all SparseML modifier-level logs to sparse_logs/22-01-2024_22.33.48.log


Training Loss: 0.5076 Acc: 0.9288
Training Loss: 0.0870 Acc: 0.9885
Training Loss: 0.0576 Acc: 0.9912
Training Loss: 0.0414 Acc: 0.9943
Training Loss: 0.0360 Acc: 0.9943
Training Loss: 0.0309 Acc: 0.9957
Training Loss: 0.0291 Acc: 0.9964
Training Loss: 0.0282 Acc: 0.9963
Training Loss: 0.0276 Acc: 0.9960
Training Loss: 0.0268 Acc: 0.9968


In [15]:
!ls -a .cache/sparsezoo/neuralmagic/

.  ..  resnet_v1-50-imagenet-base  resnet_v1-50-imagenet-pruned95_quantized
